In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [6]:
## load fips csv
fips = pd.read_csv("fips2county.tsv", sep='\t', dtype={'CountyFIPS_3': 'string'} )
fips['fips_code']= fips['CountyFIPS'].astype(str)

In [10]:
fips['fips_code'].str.len()

0       4
1       4
2       4
3       4
4       4
       ..
3137    5
3138    5
3139    5
3140    5
3141    5
Name: fips_code, Length: 3142, dtype: int64

In [16]:
fips.loc[fips['fips_code'].str.len() == 4, ['fips_code']] = '0' + fips['fips_code'][fips['fips_code'].str.len() == 4]

In [17]:
fips

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE-COUNTY,fips_code
0,1,001,Autauga,Alabama,1001,AL,AL-AUTAUGA,01001
1,1,003,Baldwin,Alabama,1003,AL,AL-BALDWIN,01003
2,1,005,Barbour,Alabama,1005,AL,AL-BARBOUR,01005
3,1,007,Bibb,Alabama,1007,AL,AL-BIBB,01007
4,1,009,Blount,Alabama,1009,AL,AL-BLOUNT,01009
...,...,...,...,...,...,...,...,...
3137,56,037,Sweetwater,Wyoming,56037,WY,WY-SWEETWATER,56037
3138,56,039,Teton,Wyoming,56039,WY,WY-TETON,56039
3139,56,041,Uinta,Wyoming,56041,WY,WY-UINTA,56041
3140,56,043,Washakie,Wyoming,56043,WY,WY-WASHAKIE,56043


In [27]:
fips.fips_code

0       01001
1       01003
2       01005
3       01007
4       01009
        ...  
3137    56037
3138    56039
3139    56041
3140    56043
3141    56045
Name: fips_code, Length: 3142, dtype: object

In [28]:
school = 'data/University of California, Riverside' # change to your school's name

In [29]:
df = pd.read_csv(school + '_clean.csv') 

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        2202 non-null   int64 
 1   current_location  2202 non-null   object
 2   current_work      2165 non-null   object
 3   previous_work     2139 non-null   object
 4   latest_Education  2202 non-null   object
 5   major             2052 non-null   object
 6   search_school     2202 non-null   object
dtypes: int64(1), object(6)
memory usage: 120.5+ KB


In [31]:
# replace wrong locations(string 'connections' in them) to nulls
no_value = df.current_location.str.contains('connections') 
df.loc[no_value,'current_location'] = np.NAN

# remove some strings not identified by geocode (!you can add more depending on your data)
l = ['Metropolitan','Area', 'Township of', 'Region', 'Greater', 'Metroplex', 'District']
df['current_location'] = df['current_location'].str.replace('|'.join(l), '', regex=True)

geolocator = Nominatim(user_agent="course_project")

In [32]:
# generate a dictionary mapping unique locations to corresponding coordinates (4 mins)
coor = {}
state = {}
county = {}
city = {}
unique_locs = df.current_location.unique()
for loc in unique_locs:
    location = geolocator.geocode(loc, timeout = 600,addressdetails=True)
    if pd.isna(location) or pd.isna(loc):
        coor[loc] = np.NAN
        state[loc] = np.NAN
        county[loc] = np.NAN
        city[loc] = np.NAN
    else:
        coor[loc] = (location.latitude, location.longitude)
        try:
            state[loc] = location.raw['address']['state']
        except:
            state[loc] = np.NAN
        try:
            county[loc] = location.raw['address']['county'].rsplit(' ',1)[0]
        except:
            county[loc] = np.NAN
        try:
            city[loc] = location.raw['address']['city']
        except:
            city[loc] = np.NAN

In [33]:
df['coordinate'] = df.apply(lambda row: coor[row.current_location], axis =1)
df['state'] = df.apply(lambda row: state[row.current_location], axis =1)
df['county'] = df.apply(lambda row: county[row.current_location], axis =1)
df['city'] = df.apply(lambda row: city[row.current_location], axis =1)

In [34]:
def mapping(row):
    ''' map state-county to fips code
    '''
    if row[['county','state']].notnull().all():
        x = fips[(fips.CountyName==row.county) & (fips.StateName==row.state)]['fips_code'].values.tolist()
        if x !=[]:
            return int(x[0])
    else:
        try:
            x = fips[(fips.CountyName==row.city) & (fips.StateName==row.state)]['fips_code'].values.tolist()
            if x !=[]:
                return int(x[0])
        except:
            pass   
    return None

In [35]:
df['fips'] = df.apply(mapping, axis=1)
df['fips']  = df['fips'] .astype(pd.Int64Dtype())

In [40]:
df = df.drop(columns=['Unnamed: 0'])

In [41]:
df

,current_location,current_work,previous_work,latest_Education,major,search_school,coordinate,state,county,city,fips
0,"Los Angeles, California, United States","Co-Founder at Black Yeti Beverage, Inc.",Chief Brand Officer & Neuromodulation Director...,2020–Present,Clinical Psychology,"University of California, Riverside","(34.0536909, -118.242766)",California,Los Angeles,Los Angeles,6037
1,"Newport Beach, California, United States",CEO at US Business Funding,Director of app only division / Small ticket E...,"University of California, Riverside",NaN,"University of California, Riverside","(33.6170092, -117.9294401)",California,Orange,Newport Beach,6059
2,"San Francisco, California, United States","CEO at Empiric, Inc.",Sales at DAQRI,ArtCenter College of Design,Product Design,"University of California, Riverside","(37.7790262, -122.419906)",California,NaN,San Francisco,6075
3,"Allen, Texas, United States",Co-Founder at TitaniusBots,Customer Representative & Sales II at Bank of ...,"University of California, Riverside",Multi/Interdisciplinary Studies,"University of California, Riverside","(33.1031744, -96.6705503)",Texas,Collin,NaN,48085
4,"Coppell, Texas, United States",Chief Executive Officer at Kings III Emergency...,"President at SoloProtect, A Division of Kings ...",University of Califonia Riverside,Mathmatics,"University of California, Riverside","(32.9552598, -97.0155703)",Texas,Dallas,NaN,48113
...,...,...,...,...,...,...,...,...,...,...,...
2197,"Littleton, Colorado, United States","Vice President, Enterprise Innovation & Digita...","Vice President, Digital Transformation Program...",1989–1991,Applied Mathematics,"University of California, Riverside","(39.613321, -105.016649)",Colorado,Arapahoe,Littleton,8005
2198,"Orange County, California, United States",Director - D2C & Global Digital Platforms at U...,Senior Manager Ecommerce at Urban Decay Cosmetics,New York University,NaN,"University of California, Riverside","(33.7500378, -117.8704931)",California,Orange,NaN,6059
2199,"Huntington Beach, California, United States",President at The Specialists in Law Firm Marke...,Director of Marketing at Deloitte & Touche LLP,1982–1984,NaN,"University of California, Riverside","(33.6783336, -118.0000166)",California,Orange,Huntington Beach,6059
2200,"San Diego, California, United States",Clinical Operations Consultant (Independent) a...,"Head, Clinical Operations at Sorrento Therapeu...",University of Illinois at Urbana-Champaign,Molecular Endrocrinology & Metabolism,"University of California, Riverside","(32.7174202, -117.1627728)",California,San Diego,San Diego,6073


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   current_location  2025 non-null   object
 1   current_work      2165 non-null   object
 2   previous_work     2139 non-null   object
 3   latest_Education  2202 non-null   object
 4   major             2052 non-null   object
 5   search_school     2202 non-null   object
 6   coordinate        2020 non-null   object
 7   state             1918 non-null   object
 8   county            1740 non-null   object
 9   city              1420 non-null   object
 10  fips              1752 non-null   Int64 
dtypes: Int64(1), object(10)
memory usage: 191.5+ KB


In [43]:
## save to a new csv file!!!
df.to_csv(school + '_with_fips.csv')

In [193]:
# bugfix (optional)
#check a specific location
location = geolocator.geocode('Frankfurt Rhine', timeout = 600)
location

Location(Frankfurter Straße, Urdenbach, Stadtbezirk 9, Düsseldorf, Nordrhein-Westfalen, 40593, Deutschland, (51.1549523, 6.8854454, 0.0))

In [194]:
#check NAN values and revise current locations to be searchable (change/add more strings to remove in l)
df[(df.coordinate.isna()) & (df.current_location.notna())]

,current_location,current_work,previous_work,latest_Education,major,search_school,coordinate,state,county,city,fips
337,"Greenville-Spartanburg-Anderson, South Carolina","President/Owner, Financial Advisor, CFP®, CRPC...","Vice President, Financial Advisor, CFP®, CRPC®...",University of South Carolina - The Moore Schoo...,Brazil/Portuguese,"University of California, Riverside",NaN,NaN,NaN,NaN,<NA>
548,Frankfurt Rhine-Main,Head of Banking and Partner Strategy and Execu...,Senior Adviser at Holland FinTech,"University of California, Riverside",German Language and Literature,"University of California, Riverside",NaN,NaN,NaN,NaN,<NA>
1001,Ezana is a startup entrepreneur currently prom...,CEO at Afri Kash,Service Management Officer at United Nations,"University of California, Riverside",Political Science,"University of California, Riverside",NaN,NaN,NaN,NaN,<NA>
1013,Laila Mickelwait is the Founder and CEO of the...,Founder/CEO at Justice Defense Fund,Founder at New Reality International,University of Southern California,Public Diplomacy,"University of California, Riverside",NaN,NaN,NaN,NaN,<NA>
1782,CEO at Cyphen Limited,CEO at Cyphen Limited,NaN,Peking University,国际经济,"University of California, Riverside",NaN,NaN,NaN,NaN,<NA>
